In [ ]:
# Install dependencies
# !pip install weaviate-client sentence-transformers pillow numpy tqdm

### Initial Setup

In [1]:
# Import libraries
import weaviate
from weaviate.classes.config import Configure, Property, DataType, Multi2VecField
import os
import base64
import json
from PIL import Image
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import time
import io

/Users/roshanakzakizadeh/Projects/multimodal_comparison/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Initialize CLIP model
clip_model = SentenceTransformer('clip-ViT-B-32')

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


### Utility Functions

In [7]:
def read_image_data(image_path):
    """Read image as base64 and PIL object, caching to avoid redundant reads"""
    with open(image_path, "rb") as img_file:
        raw_data = img_file.read()
        base64_data = base64.b64encode(raw_data).decode("utf-8")
        pil_img = Image.open(io.BytesIO(raw_data))
    return base64_data, pil_img

def process_metadata(metadata, max_chars=200):
    """Process captions from metadata"""
    captions = metadata.get("captions", [])
    concatenated = " ".join(captions)[:max_chars] if captions else "No captions available"
    return concatenated

### Schema Creation

In [8]:
def create_flickr_schema_multi2vec(client, collection_name, image_weight=0.5):
    """Create schema for multi2vec-clip"""
    if client.collections.exists(collection_name):
        client.collections.delete(collection_name)
        print(f"existing {collection_name} deleted, creating a new one.")
    
    image_weight = round(image_weight, 2)
    text_weight = round(1.0 - image_weight, 2)

    client.collections.create(
        collection_name,
        properties=[
            Property(name="image", data_type=DataType.BLOB),
            Property(name="image_id", data_type=DataType.TEXT),
            Property(name="captions", data_type=DataType.TEXT),
        ],
        vectorizer_config=[
            Configure.NamedVectors.multi2vec_clip(
                name="image_vector",
                image_fields=[Multi2VecField(name="image", weight=image_weight)],
                text_fields=[Multi2VecField(name="captions", weight=text_weight)]
            )
        ]
    )
    print(f"Created {collection_name} schema")

def create_flickr_schema_manual(client, collection_name):
    """Create schema for manual vectorization"""
    if client.collections.exists(collection_name):
        client.collections.delete(collection_name)
        print(f"existing {collection_name} deleted, creating a new one.")
        
    client.collections.create(
        collection_name,
        properties=[
            Property(name="image", data_type=DataType.BLOB),
            Property(name="image_id", data_type=DataType.TEXT),
            Property(name="captions", data_type=DataType.TEXT),
        ],
        vectorizer_config=[Configure.NamedVectors.none(name="image_vector")]
    )
    print(f"Created {collection_name} schema")

### Import Data 

In [9]:
def import_data_multi2vec(client, data_dir, collection_name, batch_size=100):
    """Import data with multi2vec-clip"""
    collection = client.collections.get(collection_name)
    images_dir = os.path.join(data_dir, "images")
    metadata_dir = os.path.join(data_dir, "metadata")
    metadata_files = [f for f in os.listdir(metadata_dir) if f.endswith('.json')]
    print(f"Importing {len(metadata_files)} images for multi2vec")

    batch_errors = []
    with collection.batch.fixed_size(batch_size=batch_size) as batch:
        for metadata_file in tqdm(metadata_files):
            try:
                with open(os.path.join(metadata_dir, metadata_file), 'r') as f:
                    metadata = json.load(f)
                
                image_path = os.path.join(images_dir, f"image_{metadata['image_id']}.jpg")
                image_data, _ = read_image_data(image_path)
                captions = process_metadata(metadata)
                
                batch.add_object(
                    properties={
                        "image": image_data,
                        "image_id": metadata["image_id"],
                        "captions": captions
                    }
                )
            except Exception as e:
                batch_errors.append(f"Error processing {metadata_file}: {str(e)}")
                continue
    if batch_errors:
        print(f"Encountered {len(batch_errors)} errors during batch import")
        for err in batch_errors[:5]:  # Print first 5 errors
            print(err)
    print("Finished multi2vec import")

def import_data_manual(client, data_dir, collection_name, image_weight = 0.5, batch_size=100):
    """Import data with manual CLIP vectorization"""
    collection = client.collections.get(collection_name)
    images_dir = os.path.join(data_dir, "images")
    metadata_dir = os.path.join(data_dir, "metadata")
    metadata_files = [f for f in os.listdir(metadata_dir) if f.endswith('.json')]
    print(f"Importing {len(metadata_files)} images for manual")

    image_weight = round(image_weight, 2)
    text_weight = round(1.0 - image_weight, 2)

    batch_errors = []
    with collection.batch.fixed_size(batch_size=batch_size) as batch:
        for metadata_file in tqdm(metadata_files):
            try:
                with open(os.path.join(metadata_dir, metadata_file), 'r') as f:
                    metadata = json.load(f)
                
                image_path = os.path.join(images_dir, f"image_{metadata['image_id']}.jpg")
                image_data, img = read_image_data(image_path)
                captions = process_metadata(metadata)
                
                image_embedding = clip_model.encode(img, convert_to_numpy=True)
                caption_list = metadata.get("captions", [])
                text_embedding = clip_model.encode(caption_list, convert_to_numpy=True).mean(axis=0) if caption_list else np.zeros_like(image_embedding)
                combined_embedding = image_weight * image_embedding + text_weight * text_embedding
                
                batch.add_object(
                    properties={
                        "image": image_data,
                        "image_id": metadata["image_id"],
                        "captions": captions
                    },
                    vector={"image_vector": combined_embedding.tolist()}
                )
                img.close() 
            except Exception as e:
                batch_errors.append(f"Error processing {metadata_file}: {str(e)}")
                continue
    if batch_errors:
        print(f"Encountered {len(batch_errors)} errors during batch import")
        for err in batch_errors[:5]:
            print(err)
    print("Finished manual import")

In [12]:

def main():
    # Connect to Weaviate
    client = weaviate.connect_to_local()

    data_dir = "flickr30k_sample"
    
    create_flickr_schema_manual(client, collection_name="Flickr30k_manual_50_50")
    create_flickr_schema_multi2vec(client, collection_name="Flickr30k_multi2vec_50_50", image_weight=0.5)

    import_data_manual(client, data_dir, collection_name="Flickr30k_manual_50_50", image_weight=0.5)
    import_data_multi2vec(client, data_dir, collection_name="Flickr30k_multi2vec_50_50")

    client.close ()


if __name__ == "__main__":
    main()

existing Flickr30k_manual_50_50 deleted, creating a new one.
Created Flickr30k_manual_50_50 schema
existing Flickr30k_multi2vec_50_50 deleted, creating a new one.
Created Flickr30k_multi2vec_50_50 schema
Importing 598 images for manual


100%|██████████| 598/598 [02:00<00:00,  4.98it/s]


Finished manual import
Importing 598 images for multi2vec


100%|██████████| 598/598 [00:35<00:00, 16.80it/s]  


Finished multi2vec import
